In [79]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd

In [67]:
headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:130.0) Gecko/20100101 Firefox/130.0',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/png,image/svg+xml,*/*;q=0.8',
        'Accept-Language': 'pt-BR,pt;q=0.8,en-US;q=0.5,en;q=0.3',
        # 'Accept-Encoding': 'gzip, deflate, br, zstd',
        'Referer': 'https://www.residentevildatabase.com/personagens/',
        'Connection': 'keep-alive',
        # 'Cookie': '_ga_DJLCSW50SC=GS1.1.1726592075.3.1.1726592152.43.0.0; _ga=GA1.2.1283352212.1726583808; _ga_D6NF5QC4QT=GS1.1.1726592077.3.1.1726592153.60.0.0; _gid=GA1.2.998084580.1726583808; __gads=ID=26adef1792c6f96f:T=1726583809:RT=1726592079:S=ALNI_MZ4dANVuS5WsvyfFwZ9t_0BhB-YRA; __gpi=UID=00000eea1583cecb:T=1726583809:RT=1726592079:S=ALNI_Mb0j0iGPJnlgh5nrG0vmeQjFnZ5LQ; __eoi=ID=584d903d1a09d505:T=1726583809:RT=1726592079:S=AA-AfjYizTMd9993gOXy6vyjYLej; FCNEC=^%^5B^%^5B^%^22AKsRol_6kz1YIGfd7lmKF-MPZWTLMbLis_jAR0bnnXAyx2TXiYS6qZZQEorLBbWVGfUnsPnaIGKKTdxB0T7202PMWsb2_9wi9WEk92vs1Q7M5bcblWzqYJpOzLAnrnyBBd4Xe2hasppm3CPllEgV1aNrrhRswzoW2A^%^3D^%^3D^%^22^%^5D^%^5D',
        'Upgrade-Insecure-Requests': '1',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'same-origin',
        'Priority': 'u=0, i',
    }

In [68]:
def get_content(url):
    resp = requests.get(url, headers=headers)
    return resp

In [69]:
def get_basic_info(soup):
    div_page = soup.find("div", class_="td-page-content")
    paragrafo = div_page.find_all("p")[1]
    ems = paragrafo.find_all("em")
    data = {}
    for i in ems:
        chave, valor, *_ = i.text.split(":")
        chave = chave.strip(" ")
        data[chave] = valor.strip(" ")
    return data

In [70]:
def get_aparicoes(soup):
    lis = (soup.find("div", class_="td-page-content").find("h4").find_next().find_all("li"))
    aparicoes = [i.text for i in lis]
    return aparicoes

In [72]:
def get_personagem_infos(url):
    resp = get_content(url)
    if resp.status_code != 200:
        print("Não foi possível obter os dados.")
        return {}
    else:
        soup = BeautifulSoup(resp.text)
        data = get_basic_info(soup)
        data["Aparicoes"] = get_aparicoes(soup)
        return data

In [73]:
def get_links():
    url = 'https://www.residentevildatabase.com/personagens/'
    resp = requests.get(url, headers=headers)
    soup_personagens = BeautifulSoup(resp.text)
    ancoras = (soup_personagens.find("div", class_="td-page-content").find_all("a"))
    links = [i["href"] for i in ancoras]
    return links

In [81]:
links = get_links()
data = []
for i in tqdm(links):
    d = get_personagem_infos(i)
    d["Link"] = i
    nome = i.strip("/").split("/")[-1].replace("-", " ").title()
    d["Nome"] = nome
    data.append(d)

 31%|███       | 51/164 [00:24<00:47,  2.39it/s]

Não foi possível obter os dados.


100%|██████████| 164/164 [01:22<00:00,  1.98it/s]


In [82]:
data

[{'Ano de nascimento': '1974 (24 anos em 1998)',
  'Tipo sanguíneo': 'AB',
  'Altura': 'Desconhecida.',
  'Peso': 'Desconhecido.',
  'Aparicoes': ['Biohazard / Resident Evil 2 (1998)',
   'Biohazard / Resident Evil 4 (2005)',
   'Biohazard / Resident Evil: The Umbrella Chronicles (2007)',
   'Biohazard / Resident Evil: The Darkside Chronicles (2009)',
   'Biohazard / Resident Evil: Operation Raccoon City (2012)',
   'Biohazard / Resident Evil: Damnation (2012)',
   'Biohazard / Resident Evil 6 (2012)',
   'Biohazard RE:2 / Resident Evil 2 (2019)',
   'Biohazard RE:4 / Resident Evil 4 (2023)'],
  'Link': 'https://www.residentevildatabase.com/ada-wong/',
  'Nome': 'Ada Wong'},
 {'Ano de nascimento': 'Desconhecido.',
  'Tipo sanguíneo': 'Desconhecido.',
  'Altura': 'Desconhecida.',
  'Peso': 'Desconhecido.',
  'Aparicoes': ['Biohazard / Resident Evil 6 (2012)'],
  'Link': 'https://www.residentevildatabase.com/adam-benford/',
  'Nome': 'Adam Benford'},
 {'Ano de nascimento': 'Desconhecido.

In [83]:
df = pd.DataFrame(data)
df

,Ano de nascimento,Tipo sanguíneo,Altura,Peso,Aparicoes,Link,Nome,de nascimento
0,1974 (24 anos em 1998),AB,Desconhecida.,Desconhecido.,"[Biohazard / Resident Evil 2 (1998), Biohazard...",https://www.residentevildatabase.com/ada-wong/,Ada Wong,NaN
1,Desconhecido.,Desconhecido.,Desconhecida.,Desconhecido.,[Biohazard / Resident Evil 6 (2012)],https://www.residentevildatabase.com/adam-benf...,Adam Benford,NaN
2,Desconhecido.,Desconhecido.,Desconhecida.,Desconhecido.,"[Biohazard / Resident Evil 7 (2017), Biohazard...",https://www.residentevildatabase.com/alan-doug...,Alan Douglas,NaN
3,1960,O,"1,83cm (em jogos antigos); 1,90cm (em RE5)","84,5kg (em jogos antigos); 90kg (em RE5)","[Biohazard / Resident Evil Clássico (1996), Bi...",https://www.residentevildatabase.com/albert-we...,Albert Wesker,NaN
4,Desconhecido.,Desconhecido.,"2,90m (aprox.)",Desconhecido.,[Biohazard / Resident Evil Village (2021)],https://www.residentevildatabase.com/alcina-di...,Alcina Dimitrescu,NaN
...,...,...,...,...,...,...,...,...
159,1962 (não confirmado).,O,"1,78cm","66,7kg","[Biohazard / Resident Evil 0 (2003), Biohazard...",https://www.residentevildatabase.com/william-b...,William Birkin,NaN
160,desconhecido.,desconhecido.,desconhecida.,desconhecido.,[Biohazard / Resident Evil: Operation Raccoon ...,https://www.residentevildatabase.com/willow-ca...,Willow Caroline Floyd,NaN
161,Desconhecido.,Desconhecido.,Desconhecido.,Desconhecido.,[BIOHAZARD / RESIDENT EVIL: Infinite Darkness ...,https://www.residentevildatabase.com/wilson-se...,Wilson Secretario Seguranca,NaN
162,1978 (20 anos em 1998),Desconhecido.,"1,63m",58kg,"[Biohazard / Resident Evil Outbreak (2003), Bi...",https://www.residentevildatabase.com/yoko-suzuki/,Yoko Suzuki,NaN


In [85]:
df.to_parquet("dados_re.parquet", index=False)

In [86]:
df_new = pd.read_parquet("dados_re.parquet")
df_new

,Ano de nascimento,Tipo sanguíneo,Altura,Peso,Aparicoes,Link,Nome,de nascimento
0,1974 (24 anos em 1998),AB,Desconhecida.,Desconhecido.,"[Biohazard / Resident Evil 2 (1998), Biohazard...",https://www.residentevildatabase.com/ada-wong/,Ada Wong,None
1,Desconhecido.,Desconhecido.,Desconhecida.,Desconhecido.,[Biohazard / Resident Evil 6 (2012)],https://www.residentevildatabase.com/adam-benf...,Adam Benford,None
2,Desconhecido.,Desconhecido.,Desconhecida.,Desconhecido.,"[Biohazard / Resident Evil 7 (2017), Biohazard...",https://www.residentevildatabase.com/alan-doug...,Alan Douglas,None
3,1960,O,"1,83cm (em jogos antigos); 1,90cm (em RE5)","84,5kg (em jogos antigos); 90kg (em RE5)","[Biohazard / Resident Evil Clássico (1996), Bi...",https://www.residentevildatabase.com/albert-we...,Albert Wesker,None
4,Desconhecido.,Desconhecido.,"2,90m (aprox.)",Desconhecido.,[Biohazard / Resident Evil Village (2021)],https://www.residentevildatabase.com/alcina-di...,Alcina Dimitrescu,None
...,...,...,...,...,...,...,...,...
159,1962 (não confirmado).,O,"1,78cm","66,7kg","[Biohazard / Resident Evil 0 (2003), Biohazard...",https://www.residentevildatabase.com/william-b...,William Birkin,None
160,desconhecido.,desconhecido.,desconhecida.,desconhecido.,[Biohazard / Resident Evil: Operation Raccoon ...,https://www.residentevildatabase.com/willow-ca...,Willow Caroline Floyd,None
161,Desconhecido.,Desconhecido.,Desconhecido.,Desconhecido.,[BIOHAZARD / RESIDENT EVIL: Infinite Darkness ...,https://www.residentevildatabase.com/wilson-se...,Wilson Secretario Seguranca,None
162,1978 (20 anos em 1998),Desconhecido.,"1,63m",58kg,"[Biohazard / Resident Evil Outbreak (2003), Bi...",https://www.residentevildatabase.com/yoko-suzuki/,Yoko Suzuki,None


In [87]:
df.to_pickle("dados_re.pkl")

In [88]:
type(df)

pandas.core.frame.DataFrame